In [1]:
from MK_MPII_loader import MPII

In [17]:
import imageio
import numpy as np
import skimage.transform
from util import generate_heatmap
from vectormath import Vector3

In [3]:
mpii = MPII(batch_size=1, task='train')

In [4]:
batch_rgb, batch_heatmap, batch_keypoint, batch_activity, batch_threshold = mpii.get_minibatch(mpii.batch_image_set())

C:\Users\Ethereum\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [11]:
rgb = batch_rgb[0]
print(rgb.shape)
imageio.imwrite('rgb.jpg', rgb)

(256, 256, 3)


C:\Users\Ethereum\Anaconda3\lib\site-packages\imageio\core\util.py:78: UserWarning: Lossy conversion from float32 to uint8, range [0, 1]
  dtype_str, out_type.__name__))


In [6]:
heatmap = batch_heatmap[0]
print(heatmap.shape, type(heatmap))
print(np.min(heatmap), np.max(heatmap))

for y in range(64):
    for x in range(64):
        heatmap[y, x, 0] = max(heatmap[y, x, :])
imageio.imwrite('heatmap.jpg', heatmap[:, :, 0])

(64, 64, 16) <class 'numpy.ndarray'>
0.0 1.0


C:\Users\Ethereum\Anaconda3\lib\site-packages\imageio\core\util.py:78: UserWarning: Lossy conversion from float32 to uint8, range [0, 1]
  dtype_str, out_type.__name__))


In [7]:
print(batch_keypoint)

[[[46.511528 34.754013]
  [33.271084 34.33032 ]
  [24.373503 29.775604]
  [24.691273 37.61395 ]
  [28.186752 39.096878]
  [29.245987 38.56726 ]
  [24.58535  33.69478 ]
  [23.42019  37.61395 ]
  [24.883875 36.556843]
  [16.236633 42.802074]
  [27.763058 33.271084]
  [21.831337 27.657133]
  [22.466879 32.741467]
  [24.373503 42.38051 ]
  [30.199299 42.486435]
  [37.93172  41.956814]]]


In [8]:
print(batch_activity)

[[1]]


In [9]:
print(batch_threshold)

[[25.6]]


In [20]:
colors = [
    Vector3(0x00, 0x00, 0x80), # Navy
    Vector3(0x00, 0x00, 0xFF), # Blue
    Vector3(0x00, 0xFF, 0x00), # Green
    Vector3(0xFF, 0xFF, 0x00), # Yellow
    Vector3(0xFF, 0x00, 0x00), # Red
    Vector3(0xFF, 0x00, 0x00), # Red
]

beautified = np.ndarray(shape=(64, 64, 3))
    
for y in range(64):
    for x in range(64):
        p = heatmap[y, x, 0] # in range [0.0, 1.0]
        index = int(p * 100) // 25 # 0 1 2 3 4
        
        t = (p - 0.25 * index)/ 0.25
        
        color = colors[index] * (1-t) + colors[index+1] * t
        beautified[y, x, :] = color

beautified = skimage.transform.resize(beautified, (256, 256))
imageio.imwrite('merged.jpg', rgb * 0.5 + beautified * 0.5 / 0xFF)

C:\Users\Ethereum\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\Ethereum\Anaconda3\lib\site-packages\imageio\core\util.py:78: UserWarning: Lossy conversion from float64 to uint8, range [0, 1]
  dtype_str, out_type.__name__))
